# ELT

Data Engineering Week 6 <br>
ETL/ELT 실습

초기 설정은 다음과 같이 해주세요!

In [59]:
# 한글말고 영어로 본인의 닉네임을 정해주세요!
name = "seungmin"

하단의 링크는 간단한 웹 페이지 입니다.<br>
http://43.200.129.41:8000/  <br>

대부분의 event를 발생시키면, log가 발생합니다. <br>
하단의 코드를 실행하면 지금까지의 log를 확인할 수 있습니다.

In [60]:
import requests, json
response = requests.get('http://43.200.129.41:8000/logs', params={"caller_name": name})
logs = json.dumps(response.json(), indent=2, ensure_ascii=False)

print(logs)

[
  {
    "id": 113,
    "user_id": null,
    "post_id": null,
    "action": "page_view",
    "details": "Viewed page: login",
    "user_agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36",
    "timestamp": "2025-11-04T09:21:27.490287"
  },
  {
    "id": 112,
    "user_id": null,
    "post_id": null,
    "action": "page_view",
    "details": "Viewed page: login",
    "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36",
    "timestamp": "2025-11-04T08:41:41.976969"
  },
  {
    "id": 111,
    "user_id": 3,
    "post_id": 1,
    "action": "like",
    "details": "Liked post 1",
    "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36",
    "timestamp": "2025-11-04T08:41:41.869411"
  },
  {
    "id": 110,
    "user_id": 3,
    "post_id": 3,
    "action": "like",
    "

만약 필요하다면, 해당 코드를 통해서 접속한 사용자의 닉네임을 확인할 수 있습니다 <br>
DB에는 user_id와 nickname이 저장되어있습니다. <br>
ex) 1 : gaeng02

In [61]:
response = requests.get('http://43.200.129.41:8000/users', params={"caller_name": name})
users = json.dumps(response.json(), indent=2, ensure_ascii=False)
print(users)

[
  {
    "id": 5,
    "nickname": "ii",
    "created_at": "2025-11-04T07:51:49.962648"
  },
  {
    "id": 4,
    "nickname": "moon",
    "created_at": "2025-11-04T07:51:31.771143"
  },
  {
    "id": 3,
    "nickname": "seungmin",
    "created_at": "2025-11-04T07:26:32.181087"
  },
  {
    "id": 2,
    "nickname": "test1234",
    "created_at": "2025-11-04T02:13:15.097616"
  },
  {
    "id": 1,
    "nickname": "gaeng02",
    "created_at": "2025-10-28T09:05:32.352465"
  }
]


이제 다음과 같은 Task를 수행하면 됩니다.

## Task 1. log 확인
log안에 어떠한 event에 로그가 발생하는지 확인하세요.

정답 예시 : event의 종류는 a, b, c, d, e가 있다.

어떤 페이지를 보았는지, 로그인을 했는지 로그아웃을 했는지, 어떤 포스트에 하트를 눌렀는지, 어떤 포스트를 눌렀는지 등등

## Task 2. 각 사용자의 마지막 접속 시간
user db안에는 현재 created_at만 저장되어있는데, last_updated_at에 대해서도 정리하시오.

In [62]:
users = requests.get('http://43.200.129.41:8000/users', params={"caller_name": name}).json()
logs  = requests.get('http://43.200.129.41:8000/logs',  params={"caller_name": name}).json()

last_access = {u["id"]: None for u in users}

for l in logs:
    uid = l.get("user_id")
    ts  = l.get("timestamp")
    if uid in last_access and ts:
        if last_access[uid] is None or ts > last_access[uid]:
            last_access[uid] = ts

for u in users:
    u["last_updated_at"] = last_access[u["id"]]

data = json.dumps(users, indent=2, ensure_ascii=False)
print(data)


[
  {
    "id": 5,
    "nickname": "ii",
    "created_at": "2025-11-04T07:51:49.962648",
    "last_updated_at": "2025-11-04T07:51:50.029968"
  },
  {
    "id": 4,
    "nickname": "moon",
    "created_at": "2025-11-04T07:51:31.771143",
    "last_updated_at": "2025-11-04T07:51:46.229767"
  },
  {
    "id": 3,
    "nickname": "seungmin",
    "created_at": "2025-11-04T07:26:32.181087",
    "last_updated_at": "2025-11-04T08:41:41.869411"
  },
  {
    "id": 2,
    "nickname": "test1234",
    "created_at": "2025-11-04T02:13:15.097616",
    "last_updated_at": "2025-11-04T02:13:18.936928"
  },
  {
    "id": 1,
    "nickname": "gaeng02",
    "created_at": "2025-10-28T09:05:32.352465",
    "last_updated_at": "2025-11-04T06:05:01.848586"
  }
]


## Task 3. 각 사용자의 행동 패턴
본인의 행동 패턴만 수집하여 하나의 파일로 정리할 수 있도록 하시오. <br>
ex) gaeng02는 user_id가 1 이므로, user_id가 1일 때의 데이터만 모아서 수집할 수 있도록 하시오. <br>

특정 user_id에 대해서 수집할 수 있도록 코드를 작성하세요.

In [63]:
def user_flow(user_id: int):

    logs = requests.get('http://43.200.129.41:8000/logs', params={"caller_name": name}).json()

    user_logs = [log for log in logs if log.get("user_id") == user_id]

    file_name = f"user_{user_id}_logs.json"

    with open(file_name, "w", encoding="utf-8") as f:
        json.dump(user_logs, f, indent=2, ensure_ascii=False)
    return user_logs

user_flow(3)

[{'id': 111,
  'user_id': 3,
  'post_id': 1,
  'action': 'like',
  'details': 'Liked post 1',
  'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36',
  'timestamp': '2025-11-04T08:41:41.869411'},
 {'id': 110,
  'user_id': 3,
  'post_id': 3,
  'action': 'like',
  'details': 'Liked post 3',
  'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36',
  'timestamp': '2025-11-04T08:41:41.289813'},
 {'id': 109,
  'user_id': 3,
  'post_id': 2,
  'action': 'like',
  'details': 'Liked post 2',
  'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36',
  'timestamp': '2025-11-04T08:41:39.896989'},
 {'id': 108,
  'user_id': 3,
  'post_id': 4,
  'action': 'like',
  'details': 'Liked post 4',
  'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Ge

## Task 4. 각 포스트의 조회수, 하트 수 확인
각 post_id마다 조회수, 하트 수를 확인해서 정리하시오. <br>

In [64]:
logs = requests.get('http://43.200.129.41:8000/logs', params={"caller_name": name}).json()

result = {}

for log in logs:
    post_id = log.get("post_id")
    event = log.get("action")

    if not post_id:
          continue

    if post_id not in result:
          result[post_id] = {"views": 0, "likes": 0}

    if event == "post_click":
          result[post_id]["views"] += 1
    elif event == "like":
          result[post_id]["likes"] += 1

data = json.dumps(result, indent=2, ensure_ascii=False)
print(data)



{
  "1": {
    "views": 1,
    "likes": 1
  },
  "3": {
    "views": 1,
    "likes": 1
  },
  "2": {
    "views": 2,
    "likes": 1
  },
  "4": {
    "views": 1,
    "likes": 1
  },
  "5": {
    "views": 2,
    "likes": 1
  },
  "6": {
    "views": 5,
    "likes": 5
  }
}


## Task 5 : 각 날짜 별 누가 접속했는지
한국시간 별로, 각 날짜에 누가 접속했는지 확인해보시오.

In [65]:
from datetime import datetime, timedelta

users = requests.get('http://43.200.129.41:8000/users', params={"caller_name": name}).json()
user_id = {u["id"]: u["nickname"] for u in users}

logs = requests.get('http://43.200.129.41:8000/logs', params={"caller_name": name}).json()

result = {}

for log in logs:
    ts = log.get("timestamp")
    uid = log.get("user_id")

    if not ts or not uid:
        continue
    dt = datetime.fromisoformat(ts)
    dt_kst = dt + timedelta(hours=9)
    date = dt_kst.date().isoformat()

    nickname = user_id.get(uid, f"user_{uid}")

    if date not in result:
        result[date] = set()
    result[date].add(nickname)

final_result = {
    date: {
        "count": len(users),
        "users": sorted(list(users))
    }
    for date, users in result.items()
}

data = json.dumps(final_result, indent=2, ensure_ascii=False)
print(data)


{
  "2025-11-04": {
    "count": 5,
    "users": [
      "gaeng02",
      "ii",
      "moon",
      "seungmin",
      "test1234"
    ]
  },
  "2025-10-30": {
    "count": 1,
    "users": [
      "gaeng02"
    ]
  },
  "2025-10-28": {
    "count": 1,
    "users": [
      "gaeng02"
    ]
  }
}


## 추가로,
중요한 것은 다음과 같은 내용을 확인하세요.

1. 몇 분에 한번씩 업데이트할 건지, 고민해보세요.
2. logs는 계속 생성됩니다. <br> 내가 갖고 있는 데이터 이후로 수집하고, update도 본인이 이미 처리한 것 이후부터 처리해야합니다.
3. 모든 Task에 대해서 문제가 발생했을 경우에도 문제없이 코드가 완료될 수 있도록 처리할 수 있는지 확인해보세요.


다음 주에 다 같이 모여서 동시에 코드를 실행해볼 예정입니다. <br>
다 동일한 로그로 결과를 실행하기 때문에 결과가 동일해야겠죠?


1. 물론 어떤 분야냐에 따라 다르겠지만 보편적인 경우 로그 데이터는 너무 자주 수집하면 서버에 가해지는 부하가 크기 때문에 5-10분 사이로 업데이트 하면 좋을 것 같다.
2. 자기가 마지막으로 수집한 시각을 DB에 저장해두고 다음에 데이터를 처리할 때는 그 이후의 데이터만 필터링하여 수집하면 될 것 같다.
3. task 코드를 try except 코드로 감싸서 만약에 오류가 발생해도 프로그램은 멈추지 않고 error log에 찍어주는 식으로 코딩을 하면 좋을 것 같다.

